In [1]:
# project
import os
import sys 
project_dir = os.path.join(os.path.expanduser('~'), 'git', 'diffsim-sysid')
sys.path.insert(1, project_dir)
# math
import numpy as np
# physics
import openmesh
import warp as wp
import warp.sim as wps
from src.warp_utils import render_usd
# config
from src.config import cfg
# dataset
from src.dataset import load_train_data
from src.data.visualize import print_example

# setup
wp.init()

2024-04-05 16:23:19.638560: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2 AVX AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


Warp 1.0.2 initialized:
   CUDA Toolkit 11.5, Driver 11.4
   Devices:
     "cpu"      : "x86_64"
     "cuda:0"   : "NVIDIA GeForce GTX 1050 Ti with Max-Q Design" (4 GiB, sm_61, mempool enabled)
   Kernel cache:
     /home/jonathan/.cache/warp/1.0.2


### Config

In [2]:
# fill in your directory set up here
config_fp = os.path.join(project_dir, f'cfg/templates/comp-movi_a.yaml')
cfg.merge_from_file(config_fp)
cfg.DATA.path = os.path.join(project_dir, 'data/sets/')
print(cfg)

DATA:
  fps: 12
  fpv: 24
  height: 256
  max_objs: 10
  path: /home/jonathan/git/diffsim-sysid/data/sets/
  set: movi_a
  width: 256
EVAL:
  OUTPUT:
    FN:
      config: config.yaml
      log: log.txt
      pred: pred.csv
    path: 
  PARAM:
    DENSITY:
      include: True
      prior: 1.35
    FRICTION:
      include: True
      prior: 0.5
    RESTITUTION:
      include: True
      prior: 0.5
SYS_ID:
  GEOM:
    include: True
    spm: 100
  OPTIM:
    beta1: 0.9
    beta2: 0.999
    decay: 0.0
    lr: 0.01
    optim: adam
  VIS:
    include: True
    spp: 4
    weight: 1.0
  iter: 4
  rand: False


### Data

In [3]:
train_ds, ds_info = load_train_data(cfg)
sample = next(iter(train_ds))

vid_id = sample['metadata']['video_name']
obj_count = sample['metadata']['num_instances']
print(f'Video: {vid_id}, Num objects: {obj_count}')
# print_example(example, ds_info)

2024-04-05 16:23:21.974329: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:981] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2024-04-05 16:23:22.000935: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:981] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2024-04-05 16:23:22.001302: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:981] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2024-04-05 16:23:22.001920: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2 AVX AVX2 FMA
To enable them in other operation

Video: b'1680', Num objects: 8


### Scene

In [129]:
# simulation parameters
SIM_DURATION = 2.0
# control frequency
FPS = 12
FRAME_DT = 1.0 / float(FPS)
FRAME_STEPS = int(SIM_DURATION / FRAME_DT)
# sim frequency
SIM_FPS = 240
SIM_SUBSTEPS = int(SIM_FPS / FPS)
SIM_STEPS = FRAME_STEPS * SIM_SUBSTEPS
SIM_DT = FRAME_DT / SIM_SUBSTEPS

# world parameters
UP = (0, 0, 1)
GRAVITY = -10.0

# contact parameters
KE = 1000.0
KD = 10.0
KF = 10.0

In [130]:
# ground parameters
CLEVR_GROUND_MU = 0.3
CLEVR_GROUND_RESTITUTION = 0.5

# object parameters
CLEVR_SHAPE_NAMES = ("cube", "cylinder", "sphere")
CLEVR_SIZE_NAMES = ('small', 'large')
CLEVR_SIZES = (0.7, 1.4)
CLEVR_DENSITIES = (2.7, 1.1)

CLEVR_SHAPE_OBJ_FPS = {
    'cube': os.path.join(project_dir, 'assets', 'cube', 'collision_geometry.obj'),
    'cylinder': os.path.join(project_dir, 'assets', 'cylinder', 'collision_geometry.obj'),
    'sphere': os.path.join(project_dir, 'assets', 'sphere', 'collision_geometry.obj')
}

CLEVR_SHAPE_BASE_MASS = {
    'cube': 0.9936,
    'cylinder': 0.7808,
    'sphere': 0.5225
}

CLEVR_SHAPE_BASE_INERTIA = {
    'cube': [
        [0.1643, 0.0, 0.0],
        [0.0, 0.1643, 0.0],
        [0.0, 0.0, 0.1643]
    ],
    'cylinder': [
        [0.1131, 0.0, 0.0],
        [0.0, 0.1131, 0.0],
        [0.0, 0.0, 0.0971]
    ],
    'sphere': [
        [0.0522, 0.0, 0.0],
        [0.0, 0.0522, 0.0],
        [0.0, 0.0, 0.0522]
    ]
}

### Physics

In [131]:
def build_phys(device, sample):
    builder = wps.ModelBuilder(up_vector=wp.vec3(UP), gravity=GRAVITY)

    # ground
    builder.set_ground_plane(ke=KE, 
                             kd=KD, 
                             kf=KF,
                             mu=CLEVR_GROUND_MU, 
                             restitution=CLEVR_GROUND_RESTITUTION)

    for obj_idx in range(obj_count):
        obj_class_idx = sample['instances']['shape_label'][obj_idx]
        obj_class = CLEVR_SHAPE_NAMES[obj_class_idx]

        obj_mesh_fp = CLEVR_SHAPE_OBJ_FPS[obj_class]
        obj_base_mass = CLEVR_SHAPE_BASE_MASS[obj_class]
        obj_base_inertia = CLEVR_SHAPE_BASE_INERTIA[obj_class]

        obj_scale_idx = sample['instances']['size_label'][obj_idx]
        obj_scale = [CLEVR_SIZES[obj_scale_idx]] * 3

        obj_pos = sample['instances']['positions'][obj_idx][0]
        obj_rot = sample['instances']['quaternions'][obj_idx][0]
        # w,i,j,k -> i,j,k,w
        obj_rot = np.concatenate([obj_rot[1:], obj_rot[:1]])

        obj_lin_vel = sample['instances']['velocities'][obj_idx][0]
        obj_ang_vel = sample['instances']['angular_velocities'][obj_idx][0]

        obj_fric = sample['instances']['friction'][obj_idx]
        obj_restitution = sample['instances']['restitution'][obj_idx]

        obj_material = sample['instances']['material_label'][obj_idx]
        obj_density = CLEVR_DENSITIES[obj_material]


        # read mesh
        m = openmesh.read_trimesh(obj_mesh_fp)
        mesh_points = np.array(m.points())
        mesh_indices = np.array(m.face_vertex_indices(), dtype=np.int32).flatten()
        mesh = wps.Mesh(vertices=mesh_points, 
                        indices=mesh_indices,
                        compute_inertia=False,
                        is_solid=True)
        mesh.has_inertia = True
        mesh.I = wp.mat33(obj_base_inertia)
        mesh.mass = obj_base_mass

        # add body
        obj_body_q = wp.transform(obj_pos, obj_rot)
        obj_body_id = builder.add_body(origin=obj_body_q, name = f'obj_{obj_idx}')

        # add mesh to body
        builder.add_shape_mesh(body=obj_body_id,
                                    mesh=mesh,
                                    scale=obj_scale,
                                    density=obj_density,
                                    mu=obj_fric,
                                    ke=KE,
                                    kd=KD,
                                    kf=KF,
                                    restitution=obj_restitution,
                                    is_solid=True,
                                    has_ground_collision=True)
        
        obj_body_qd = np.concatenate([obj_ang_vel, obj_lin_vel]).tolist()
        builder.body_qd[obj_body_id] = obj_body_qd

    # model and states
    model = builder.finalize(device, requires_grad=False)
    states = [model.state(requires_grad=False) for _ in range(SIM_STEPS+1)]

    return model, states
        

def run_phys(model, integrator, states):
    # computation graph
    tape = wp.Tape()
    with tape:
        for i in range(SIM_STEPS):
            states[i].clear_forces()
            wp.sim.collide(model, states[i])
            integrator.simulate(model, states[i], states[i + 1], SIM_DT)

    return tape

In [132]:
phys_device = wp.get_cuda_device()
phys_model, phys_states = build_phys(phys_device, sample)
phys_integrator = wp.sim.SemiImplicitIntegrator()
phys_tape = run_phys(phys_model, phys_integrator, phys_states)

#phys_stage = f'movi_a-{vid_id}.usd'
#render_usd(phys_stage, phys_model, phys_states, SIM_DURATION, SIM_DT)

100%|██████████| 481/481 [00:01<00:00, 459.00it/s]

Saved the USD stage file at `/home/jonathan/git/diffsim-sysid/examples/physics/movi_a-b'1680'.usd`
